# Optimal Solution by Integer Programming

by [Lei You](http://user.it.uu.se/~leiyo378)

In this document, we solve the flexible TTI allocation problem to optimum, by using integer programming solver gurobi.
The formulation of the original problem is as below.

$$
\begin{align}
\max_{\mathbf{x}} \quad & \sum_{b\in\mathcal{B}}\sum_{k\in\mathcal{K}^{(c)}} r_{b,k}x_{b,k} \\
s.t. \quad & \sum_{b\in\mathcal{B}}r_{b,k}x_{b,k}\geq q_{k} \quad k\in\mathcal{K}^{(\ell)} \\
           & \sum_{b\in\mathcal{B}}\sum_{k\in\mathcal{K}}a_{b,i}x_{b,k}\leq 1 \quad i\in\mathcal{I} \\
           & x_{b,k}\in\{0,1\}\quad b\in\mathcal{B},~k\in\mathcal{K}
\end{align}
$$

We have $x_{b,k}=1$ if and only if physical resource block (PRB) $b$ is allocated to service $k$.

Recall that $\tau_k$ is the maximum tolerant latency of service $k$, and $d_b$ is the end time of the PRB $b$. The constraints for the latency is imposed, by letting $r_{b,k}$ follow the rule:
$$
r_{b,k}=\left\{
\begin{array}{ll}
0 & \text{if } \tau_k-d_b<0 \\
\text{capacity} & \text{otherwise}
\end{array}
\right.
$$

Now let's start :)

In [294]:
import sys
import scipy.io
import numpy
import math
import csv

from gurobipy import *

# set the directory path
import os
folder_name = os.getcwd()

The sets $\mathcal{B}$, $\mathcal{K}^{(\ell)}$, $\mathcal{K}^{(c)}$, $\mathcal{K}$, and $\mathcal{I}$ are read by the following code.

In [295]:
# # set of physical layer blocks (PRBs)
with open('B.csv', 'rb') as f:
    B_csv = csv.reader(f)
    B = list(B_csv)
    B = [item for sublist in B for item in sublist] # flatten list
    B = map(int, map(float, B)) # convert to int
    
# # set of latency services    
with open('Kl.csv', 'rb') as f:
    Kl_csv = csv.reader(f)
    Kl = list(Kl_csv)
    Kl = [item for sublist in Kl for item in sublist] # flatten list
    Kl = map(int, map(float, Kl)) # convert to int
    
# # set of capacity services    
with open('Kc.csv', 'rb') as f:
    Kc_csv = csv.reader(f)
    Kc = list(Kc_csv)
    Kc = [item for sublist in Kc for item in sublist] # flatten list
    Kc = map(int, map(float, Kc)) # convert to int
    
# # set of all services
K = Kl + Kc

# # set of resource units (RUs)
with open('I.csv', 'rb') as f:
    I_csv = csv.reader(f)
    I = list(I_csv)
    I = [item for sublist in I for item in sublist] # flatten list
    I = map(int, map(float, I)) # convert to int

The parameters $\mathbf{r}$, $\mathbf{q}$, and $\mathbf{a}$ are read below.

In [296]:
# # matrix r
with open('r.csv', 'rb') as f:
    r_csv = csv.reader(f)
    r = list(r_csv)
    r = [ map(int,map(float,x)) for x in r] # convert to int

# # vector q, only for Kl
with open('q.csv', 'rb') as f:
    q_csv = csv.reader(f)
    q = list(q_csv)
    q = [item for sublist in q for item in sublist] # flatten list
    q = map(int, map(float, q)) # convert to int
    
# # matrix a
with open('a.csv', 'rb') as f:
    a_csv = csv.reader(f)
    a = list(a_csv)
    a = [ map(int,map(float,x)) for x in a] # convert to int

The model is created below in gurobi.

In [297]:
model = Model('Interger Programming')
model.modelSense = GRB.MAXIMIZE
# model.setParam('OutputFlag', False) # slience output

The variables $x_{b,k}$ ($b\in\mathcal{B}$, $k\in\mathcal{K}$) are added to the model by the following loop. And $\mathbf{x}$ is imposed to be binary.

In [298]:
x = []
for b in B:
    x_b = []
    for k in K:
#         x_b.append(model.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1))
        x_b.append(model.addVar(vtype=GRB.BINARY))
    x.append(x_b)
    
model.update()

The first constraint, $\sum_{b\in\mathcal{B}}r_{b,k}x_{b,k}\geq q_{k}$ for all services $k\in\mathcal{K}^{(\ell)}$, is added below.

In [299]:
for k in Kl:
    model.addConstr( sum(r[b][k]*x[b][k] for b in B) >= q[k] )
model.update()

The second constraint, $\sum_{b\in\mathcal{B}}\sum_{k\in\mathcal{K}}a_{b,i}x_{b,k}\leq 1$ for all resource units $i\in\mathcal{I}$, is added as follows.

In [300]:
for i in I:
    model.addConstr( sum(a[b][i]*x[b][k] for k in K for b in B ) <= 1 )
model.update()

The objective function $\sum_{b\in\mathcal{B}}\sum_{k\in\mathcal{K}^{(c)}} r_{b,k}x_{b,k}$, is added below.

In [301]:
model.setObjective(
    sum( r[b][k]*x[b][k] for k in Kc for b in B )
)

Now the problem can be solved by the solver:

In [302]:
model.optimize()

Optimize a model with 92 rows, 2530 columns and 10695 nonzeros
Variable types: 0 continuous, 2530 integer (2530 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+02, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+02]
Found heuristic solution: objective 770
Presolve removed 31 rows and 2088 columns
Presolve time: 0.02s
Presolved: 61 rows, 442 columns, 2092 nonzeros
Found heuristic solution: objective 895.0000000
Variable types: 0 continuous, 442 integer (441 binary)

Root relaxation: objective 1.998815e+03, 168 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1998.81520    0   15  895.00000 1998.81520   123%     -    0s
H    0     0                    1676.0000000 1998.81520  19.3%     -    0s
     0     0 1977.66643    0   35 1676.00000 1977.66643  18.0%     -    0s
H    0     0       

In [303]:
def isFeasible(x):
    if (numpy.dot( numpy.dot(numpy.matrix(a).transpose(), 
                             numpy.matrix(x)), numpy.ones(len(K)) ) 
        > numpy.ones(len(I))).tolist()[0].count(True)>0:
        return False
    if (numpy.dot(numpy.multiply(numpy.matrix(r),numpy.matrix(x))[:,0:len(Kl)].transpose(),
                  numpy.ones(len(B)))<numpy.matrix(q)).tolist()[0].count(True)>0:
        return False
    return True

In [304]:
numpy.savetxt("opt_x.csv", [ [ int(x[b][k].x) for k in K] for b in B ], delimiter=",")
numpy.savetxt("lp_x.csv", [ [ x[b][k].x for k in K] for b in B ], delimiter=",")

In [305]:
# if conflict_PRBs[b1][b2] == True, then the PRBs b1 b2 cannot be used simultaneously
with open('conflict_PRB.csv', 'rb') as f:
    conflict_PRB_csv = csv.reader(f)
    conflict_PRB = list(conflict_PRB_csv)
    conflict_PRB = [ map(lambda y: int(float(y))==1, t) for t in conflict_PRB]

In [306]:
x_count = [ [ x[b][k].x for b in B] for k in K ]
sol_x = [ [ 0 for k in K ] for b in B ]
collision = [ False for b in B ]
priority = [ [i[0] for i in sorted(enumerate(x_count[k]), key=lambda y:y[1],reverse=True)] for k in K ]


best = 0
for k in range(400):
    Kl_rand = numpy.random.permutation(Kl)
    Kc_rand = numpy.random.permutation(Kc)
    B_rand = numpy.random.permutation(B)
    
    for k in Kl_rand:
        service_bit = sum(r[b][k]*sol_x[b][k] for b in B)
        while service_bit < q[k] and collision.count(False)>0: 
            for pos in range(len(priority[k])):
                b = priority[k][pos]
                if collision[b] == False: 
                    sol_x[b][k] = 1
                    service_bit += r[b][k]*sol_x[b][k]
                    for p in B: # set all PRB overlapping with b to be in collision
                        if conflict_PRB[b][p] == True:
                            collision[p] = True
                    break
                    
    for k in Kc_rand:
        for b in B_rand:
            if collision[b] == False and x[b][k].x >= 0.5:
                sol_x[b][k]=1
                for p in B:
                    if conflict_PRB[b][p] == True:
                        collision[p] = True
                        
    while collision.count(False)>0:
        for k in Kc_rand:
            for pos in range(len(priority[k])):
                b = priority[k][pos]
                if collision[b] == False:
                    sol_x[b][k] = 1
                    for p in B:
                        if conflict_PRB[b][p] == True:
                            collision[p] = True
                    break
                        
    obj = sum( r[b][k]*int(sol_x[b][k]) for k in Kc for b in B )
    if obj > best:
        best = obj
                
    
print best, isFeasible(sol_x)


1911 True


In [307]:
import pandas
heuristic_x = sol_x
print 'allocated blocks number:', [ sum(numpy.matrix(heuristic_x).transpose()[k].tolist()[0]) for k in K ], 'obj=', sum(r[b][k]*heuristic_x[b][k] for k in Kc for b in B)
indices = [ [ [i,k] for i,t in enumerate(numpy.matrix(heuristic_x)[:,k].transpose().tolist()[0]) if t==1 ] for k in K]
pandas.DataFrame(indices)

allocated blocks number: [2, 3, 3, 2, 6, 2, 0, 3, 0, 0] obj= 1911


,0,1,2,3,4,5
0,"[0, 0]","[40, 0]",None,None,None,None
1,"[15, 1]","[25, 1]","[30, 1]",None,None,None
2,"[20, 2]","[210, 2]","[250, 2]",None,None,None
3,"[185, 3]","[217, 3]",None,None,None,None
4,"[35, 4]","[89, 4]","[195, 4]","[196, 4]","[243, 4]","[244, 4]"
5,"[5, 5]","[50, 5]",None,None,None,None
6,None,None,None,None,None,None
7,"[11, 7]","[45, 7]","[59, 7]",None,None,None
8,None,None,None,None,None,None
9,None,None,None,None,None,None
